<a href="https://colab.research.google.com/github/hwangho-kim/Utility-OAC/blob/main/%EB%8C%80%ED%99%94%ED%98%95_%EC%8A%B5%EA%B3%B5%EA%B8%B0%EC%84%A0%EB%8F%84_(%EC%9E%AC%EC%9E%91%EC%84%B1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import plotly.graph_objects as go
import numpy as np
import pandas as pd
import io # 문자열로부터 CSV를 읽기 위한 모듈
import os # 실제 파일 시스템 상호작용을 위한 모듈
import traceback # 예외 발생 시 상세 정보 출력을 위함

# 1. 기본 상수 정의
C_PA = 1.006  # 건공기의 정압비열 (kJ/kg°C)
C_PW = 1.86   # 수증기의 정압비열 (kJ/kg°C)
H_FG0 = 2501  # 0°C에서의 물의 증발잠열 (kJ/kg)
P_ATM = 101.325 # 표준 대기압 (kPa)

# 2. 습공기 상태량 계산 함수
def get_saturation_vapor_pressure_kPa(temp_c):
    if pd.isna(temp_c) or temp_c < -60: return np.nan
    return 0.61094 * np.exp((17.625 * temp_c) / (temp_c + 243.04))

def get_actual_vapor_pressure_kPa(temp_c, rh_percent):
    if pd.isna(temp_c) or pd.isna(rh_percent): return np.nan
    p_ws = get_saturation_vapor_pressure_kPa(temp_c)
    if pd.isna(p_ws): return np.nan
    return (rh_percent / 100.0) * p_ws

def get_humidity_ratio_kg_kg(temp_c, rh_percent, p_atm_kPa=P_ATM):
    if pd.isna(temp_c) or pd.isna(rh_percent): return np.nan
    p_w = get_actual_vapor_pressure_kPa(temp_c, rh_percent)
    if pd.isna(p_w): return np.nan
    denominator = p_atm_kPa - p_w
    if denominator <= 1e-9: # 분모가 0에 가까운 경우
        p_ws_check = get_saturation_vapor_pressure_kPa(temp_c) # 포화 수증기압 다시 확인
        if not pd.isna(p_ws_check) and abs(p_w - p_ws_check) < 1e-6 and rh_percent > 99.9: # 100% RH 근처
             p_w_adjusted = p_ws_check * 0.99999 # 미세 조정
             denominator_adjusted = p_atm_kPa - p_w_adjusted
             if denominator_adjusted <= 1e-9: return float('inf')
             return 0.621945 * p_w_adjusted / denominator_adjusted
        return float('inf') # 그 외 극단적인 경우
    return 0.621945 * p_w / denominator

def get_enthalpy_kJ_kg(temp_c, rh_percent, p_atm_kPa=P_ATM):
    if pd.isna(temp_c) or pd.isna(rh_percent): return np.nan
    w = get_humidity_ratio_kg_kg(temp_c, rh_percent, p_atm_kPa)
    if pd.isna(w) or w == float('inf'): return np.nan
    return C_PA * temp_c + w * (H_FG0 + C_PW * temp_c)

def get_temp_from_enthalpy_humidity_ratio(h_kJ_kg, w_kg_kg):
    if pd.isna(h_kJ_kg) or pd.isna(w_kg_kg): return np.nan
    denominator = C_PA + w_kg_kg * C_PW
    if abs(denominator) < 1e-9: return float('nan')
    return (h_kJ_kg - w_kg_kg * H_FG0) / denominator

# 3. Plotly용 습공기선도 배경 그리기 함수
def add_psychrometric_background_to_fig(fig, temp_range_c, w_range_kg_kg):
    temps_c_plot = np.linspace(temp_range_c[0], temp_range_c[1], 100)

    # 상대습도 100% (포화선)
    w_saturated = np.array([get_humidity_ratio_kg_kg(t, 100) for t in temps_c_plot])
    valid_sat_indices = ~np.isnan(w_saturated) & ~np.isinf(w_saturated) & (w_saturated <= w_range_kg_kg[1]) & (w_saturated >= w_range_kg_kg[0])
    fig.add_trace(go.Scatter(
        x=temps_c_plot[valid_sat_indices], y=w_saturated[valid_sat_indices],
        mode='lines', name='상대습도 100%', line=dict(color='black', width=2), # 더 두껍게
        hoverinfo='skip', showlegend=True, legendgroup="background"
    ))

    # 기타 상대습도선
    rh_curves_to_plot = [20, 40, 60, 80]
    for rh_val in rh_curves_to_plot:
        w_rh_plot = np.array([get_humidity_ratio_kg_kg(t, rh_val) for t in temps_c_plot])
        valid_rh_indices = ~np.isnan(w_rh_plot) & ~np.isinf(w_rh_plot) & (w_rh_plot <= w_range_kg_kg[1]) & (w_rh_plot >= w_range_kg_kg[0])
        if np.any(valid_rh_indices):
            fig.add_trace(go.Scatter(
                x=temps_c_plot[valid_rh_indices], y=w_rh_plot[valid_rh_indices],
                mode='lines', name=f'상대습도 {rh_val}%', line=dict(color='rgba(0,0,255,0.6)', width=1, dash='dash'), # 색상 및 투명도 조정
                hoverinfo='skip', showlegend=True, legendgroup="background"
            ))

    # 엔탈피선
    h_min_approx = get_enthalpy_kJ_kg(temp_range_c[0], 20 if temp_range_c[0] > 0 else 80)
    h_max_approx = get_enthalpy_kJ_kg(temp_range_c[1], 100)
    if pd.isna(h_min_approx) or h_min_approx == float('inf'): h_min_approx = 0
    if pd.isna(h_max_approx) or h_max_approx == float('inf'):
        h_max_approx = get_enthalpy_kJ_kg(temp_range_c[1], 80, P_ATM)
        if pd.isna(h_max_approx) or h_max_approx == float('inf'): h_max_approx = 120

    enthalpy_lines_values = np.unique(np.linspace(round(max(0,h_min_approx) / 10) * 10, round(h_max_approx / 10) * 10, 10).astype(int))
    enthalpy_lines_values = [h for h in enthalpy_lines_values if not pd.isna(h) and h != float('inf') and h != float('-inf') and h >=0]

    for h_val in enthalpy_lines_values:
        w_for_h_line = np.linspace(w_range_kg_kg[0], w_range_kg_kg[1], 50)
        temps_for_h_line = np.array([get_temp_from_enthalpy_humidity_ratio(h_val, w) for w in w_for_h_line])
        valid_h_points_indices = ~np.isnan(temps_for_h_line) & \
                                 (temps_for_h_line >= temp_range_c[0]) & (temps_for_h_line <= temp_range_c[1]) & \
                                 (w_for_h_line >= w_range_kg_kg[0]) & (w_for_h_line <= w_range_kg_kg[1])
        if np.sum(valid_h_points_indices) > 1:
            temps_plot_h = temps_for_h_line[valid_h_points_indices]
            w_plot_h = w_for_h_line[valid_h_points_indices]
            fig.add_trace(go.Scatter(
                x=temps_plot_h, y=w_plot_h, mode='lines',
                name=f'엔탈피={h_val}kJ/kg', line=dict(color='rgba(0,128,0,0.6)', width=1, dash='dot'), # 색상 및 투명도 조정
                hoverinfo='skip', showlegend=False, legendgroup="background"
            ))
            if len(temps_plot_h) > 0: # 엔탈피 값 주석
                idx_annot = len(temps_plot_h) // 2 # 중간 지점 또는 첫 지점
                text_x, text_y = temps_plot_h[idx_annot], w_plot_h[idx_annot]
                # 차트 경계 내에 있는지 확인 후 주석 추가
                if temp_range_c[0] <= text_x <= temp_range_c[1] and \
                   w_range_kg_kg[0] <= text_y <= w_range_kg_kg[1]:
                    fig.add_annotation(x=text_x, y=text_y, text=f"{h_val}", showarrow=False,
                                       font=dict(color="green", size=8), xanchor="center", yanchor="middle",
                                       bgcolor="rgba(255,255,255,0.5)", borderpad=2, textangle=-30)

# --- 파일 및 컬럼 처리 함수 ---
STAGE_MAPPING_DEFINITIONS = [
    {'id': 'PHC', 'display_name': "1. PH/C (예열)", 'stage_keywords': ["PH/C", "PREHEAT", "예열"], 'temp_keyword': "온도", 'rh_keyword': "습도", 'color': '#ff7f0e'},
    {'id': 'PCC', 'display_name': "2. PC/C (예냉)", 'stage_keywords': ["PC/C", "PRECOOL", "예냉"], 'temp_keyword': "온도", 'rh_keyword': "습도", 'color': '#2ca02c'},
    {'id': 'CC',  'display_name': "3. C/C (냉각)", 'stage_keywords': ["C/C", "COOLINGCOIL", "냉각코일", "COOLING", "COOL"], 'temp_keyword': "온도", 'rh_keyword': "습도", 'color': '#d62728'},
    {'id': 'HC',  'display_name': "4. H/C (가열/재열)", 'stage_keywords': ["H/C", "HEATINGCOIL", "재열코일", "승온", "HEATING", "HEAT", "REHEAT"], 'temp_keyword': "온도", 'rh_keyword': "습도", 'color': '#9467bd'}
]
def map_columns_from_csv(df_columns):
    mapped_stages_config = []
    print("\n컬럼 매핑 시도 중...")
    available_columns = list(df_columns)
    for stage_def in STAGE_MAPPING_DEFINITIONS:
        found_temp_col, found_rh_col = None, None
        # 온도 컬럼 찾기 (가장 점수가 높은 것 또는 첫 번째 것)
        best_temp_score, best_rh_score = -1, -1

        for col_name in available_columns:
            col_lower = col_name.lower()
            is_stage_match = any(sk.lower() in col_lower for sk in stage_def['stage_keywords'])
            if is_stage_match:
                if stage_def['temp_keyword'].lower() in col_lower:
                    # 간단하게 첫 번째 매칭 사용 (또는 더 정교한 점수 로직 추가 가능)
                    if found_temp_col is None: found_temp_col = col_name
                if stage_def['rh_keyword'].lower() in col_lower and col_name != found_temp_col:
                    if found_rh_col is None: found_rh_col = col_name

        if found_temp_col and found_rh_col:
            mapped_stages_config.append({
                'name': stage_def['display_name'], 'temp_col': found_temp_col,
                'rh_col': found_rh_col, 'color': stage_def['color']})
            print(f"  매핑 성공: {stage_def['display_name']} (온도: '{found_temp_col}', 습도: '{found_rh_col}')")
            if found_temp_col in available_columns: available_columns.remove(found_temp_col)
            if found_rh_col in available_columns: available_columns.remove(found_rh_col)
        elif found_temp_col or found_rh_col:
            print(f"  부분 매핑: {stage_def['display_name']} (온도: {found_temp_col}, 습도: {found_rh_col}) - 차트 미포함")
    if not mapped_stages_config: print("경고: 매핑 가능한 스테이지가 없습니다.")
    return mapped_stages_config

# 4. 대화형 차트 생성 함수
def create_interactive_psychrometric_chart_plotly(df, mapped_stages_config, selected_filename):
    if df.empty or not mapped_stages_config:
        print("데이터 또는 스테이지 설정이 없어 차트를 생성할 수 없습니다.")
        return

    # 차트 범위 계산
    all_temps, all_ws = [], []
    for _, row in df.iterrows():
        for stage in mapped_stages_config:
            try:
                temp, rh = row[stage['temp_col']], row[stage['rh_col']]
                if pd.isna(temp) or pd.isna(rh): continue
                all_temps.append(temp)
                w = get_humidity_ratio_kg_kg(temp, rh)
                if not pd.isna(w) and w != float('inf') and 0 <= w < 0.1: all_ws.append(w) # 유효 범위 w
            except KeyError: continue

    if not all_temps or not all_ws:
        print("차트 범위 계산을 위한 유효 데이터가 없습니다. 기본 범위를 사용합니다.")
        min_temp_data, max_temp_data, min_w_data, max_w_data = 0, 40, 0, 0.025
    else:
        min_temp_data, max_temp_data = min(all_temps), max(all_temps)
        min_w_data, max_w_data = min(all_ws), max(all_ws)

    temp_margin = max(5, (max_temp_data - min_temp_data) * 0.1)
    w_margin = max(0.002, (max_w_data - min_w_data) * 0.1)
    chart_min_temp, chart_max_temp = min_temp_data - temp_margin, max_temp_data + temp_margin * 1.5
    chart_min_w, chart_max_w = 0, min(0.035, max_w_data + w_margin * 2.0)

    w_sat_at_max_temp = get_humidity_ratio_kg_kg(chart_max_temp, 100)
    if not pd.isna(w_sat_at_max_temp) and w_sat_at_max_temp != float('inf'):
        chart_max_w = max(chart_max_w, w_sat_at_max_temp * 1.05)


    # 기본 Figure 생성 및 배경 추가
    fig = go.Figure()
    add_psychrometric_background_to_fig(fig, (chart_min_temp, chart_max_temp), (chart_min_w, chart_max_w))

    # 동적 요소(포인트, 경로)를 위한 트레이스 uid 정의
    POINTS_TRACE_UID = "dynamic-points"
    PATH_TRACE_UID = "dynamic-path"

    # 플레이스홀더 동적 트레이스 추가
    fig.add_trace(go.Scatter(uid=POINTS_TRACE_UID, x=[], y=[], mode='markers', name="공조 지점", showlegend=False))
    fig.add_trace(go.Scatter(uid=PATH_TRACE_UID, x=[], y=[], mode='lines', line=dict(color='lightgrey', width=2), name="공조 경로", showlegend=False))

    # 프레임 생성
    frames = []
    print("차트 프레임 생성 중...")
    for ts_idx, (timestamp, row_data) in enumerate(df.iterrows()): # df는 Timestamp를 인덱스로 가짐
        ts_str = timestamp.strftime('%Y-%m-%d %H:%M:%S')
        frame_points_x, frame_points_y, frame_texts, frame_customdata, frame_colors = [], [], [], [], []
        current_annotations = []

        for stage_conf in mapped_stages_config:
            temp_c, rh_p = np.nan, np.nan
            try:
                temp_c, rh_p = row_data[stage_conf['temp_col']], row_data[stage_conf['rh_col']]
            except KeyError: pass

            w, h = np.nan, np.nan
            if not (pd.isna(temp_c) or pd.isna(rh_p)):
                w = get_humidity_ratio_kg_kg(temp_c, rh_p)
                h = get_enthalpy_kJ_kg(temp_c, rh_p)

            is_valid_point = not (pd.isna(w) or pd.isna(h) or w == float('inf') or h == float('inf') or w >= chart_max_w * 1.2 or w < chart_min_w)

            if is_valid_point:
                frame_points_x.append(temp_c)
                frame_points_y.append(w)
                frame_texts.append(stage_conf['name'])
                frame_customdata.append([rh_p, h])
                frame_colors.append(stage_conf['color'])
                current_annotations.append(dict(
                    x=temp_c, y=w, text=f"<b>{stage_conf['name']}</b><br>{temp_c:.1f}°C, {rh_p:.0f}%<br>h={h:.1f}kJ/kg",
                    showarrow=True, arrowhead=1, arrowwidth=1, arrowcolor=stage_conf['color'], ax=20, ay=-30,
                    font=dict(size=9), bordercolor=stage_conf['color'], borderwidth=1, bgcolor="rgba(255,255,255,0.85)"
                ))
            else: # 유효하지 않은 포인트는 None으로 추가하여 선 연결 방지
                frame_points_x.append(None)
                frame_points_y.append(None)
                frame_texts.append(stage_conf['name'] + " (데이터 없음)") # 호버 텍스트용
                frame_customdata.append([np.nan, np.nan])
                frame_colors.append('rgba(200,200,200,0.5)')


        frames.append(go.Frame(
            name=ts_str,
            data=[ # 업데이트할 트레이스 데이터 (uid 사용)
                go.Scatter(uid=POINTS_TRACE_UID, x=frame_points_x, y=frame_points_y, text=frame_texts,
                           customdata=np.array(frame_customdata).T if frame_customdata and any(c is not None for c_list in frame_customdata for c in c_list) else np.array([]).reshape(2,0),
                           mode='markers', marker=dict(color=frame_colors, size=10, line=dict(width=1, color='DarkSlateGrey'))
                           ),
                go.Scatter(uid=PATH_TRACE_UID, x=frame_points_x, y=frame_points_y, mode='lines', line=dict(color='lightgrey', width=2))
            ],
            layout=go.Layout(annotations=current_annotations) # 현재 프레임의 주석만 설정
        ))
    print("프레임 생성 완료.")
    fig.frames = frames

    # 슬라이더 설정
    sliders = [dict(
        active=0, currentvalue={"prefix": "시간: ", "font": {"size": 14}}, pad={"t": 60, "b": 10},
        steps=[dict(label=df.index[i].strftime('%m-%d %H:%M'), method="animate", # df.index 사용
                    args=[[df.index[i].strftime('%Y-%m-%d %H:%M:%S')],
                          dict(mode="immediate", frame=dict(duration=50, redraw=True), transition=dict(duration=0))])
               for i in range(len(df))] # df.index 사용
    )]

    # 초기 레이아웃 설정
    initial_ts_str = df.index[0].strftime('%Y-%m-%d %H:%M:%S') # df.index 사용
    fig.update_layout(
        title_text=f"대화형 습공기선도: {selected_filename} - {initial_ts_str}",
        width=1200, height=800,
        xaxis=dict(title='건구온도 (°C)', range=[chart_min_temp, chart_max_temp], autorange=False, gridcolor='rgba(200,200,200,0.5)'),
        yaxis=dict(title='절대습도 (kg_water / kg_dry_air)', range=[chart_min_w, chart_max_w], autorange=False, gridcolor='rgba(200,200,200,0.5)'),
        sliders=sliders,
        legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        margin=dict(l=70, r=50, b=120, t=100), hovermode="closest",
        plot_bgcolor='rgba(245,245,245,1)' # 배경색 약간 추가
    )

    # 첫 번째 프레임의 데이터와 주석으로 초기 차트 상태 설정
    if frames:
        first_frame = frames[0]
        # data 업데이트
        for trace_idx, trace_data_in_frame in enumerate(first_frame.data):
            # fig.data의 해당 uid를 가진 트레이스를 업데이트
            # uid가 일치하는 트레이스를 찾아 업데이트
            target_trace_uid = trace_data_in_frame.uid
            for fig_trace in fig.data:
                if fig_trace.uid == target_trace_uid:
                    fig_trace.x = trace_data_in_frame.x
                    fig_trace.y = trace_data_in_frame.y
                    if hasattr(trace_data_in_frame, 'text'): fig_trace.text = trace_data_in_frame.text
                    if hasattr(trace_data_in_frame, 'customdata'): fig_trace.customdata = trace_data_in_frame.customdata
                    if hasattr(trace_data_in_frame, 'marker'): fig_trace.marker = trace_data_in_frame.marker
                    if hasattr(trace_data_in_frame, 'line'): fig_trace.line = trace_data_in_frame.line
                    break
        # layout (annotations) 업데이트
        if hasattr(first_frame.layout, 'annotations'):
            fig.layout.annotations = first_frame.layout.annotations
        else:
            fig.layout.annotations = []
    else: # 프레임이 없는 경우
        fig.layout.annotations = []


    print("차트 표시 중...")
    fig.show()

# --- 더미 CSV 데이터 ---
dummy_csv_data_ahu1_oac = """Timestamp,AHU01_PH/C_온도_입구,AHU01_PH/C_습도_입구,AHU01_PC/C_Zone1_온도,AHU01_PC/C_Zone1_습도,AHU01_CoolingCoil_Air_온도_Avg,AHU01_CoolingCoil_Air_습도_Avg,AHU01_HC_승온후_온도값,AHU01_HC_승온후_습도값
2023-07-15 10:00:00,30.0,65,27.0,75,16.0,92,21.0,68
2023-07-15 10:10:00,30.2,66,26.5,76,14.0,96,23.0,58
2023-07-15 11:00:00,31.0,65,26.0,77,12.5,98,25.5,47
2023-07-15 11:59:59,31.5,66.5,25.0,78.5,13.5,96.5,26.5,45.5
2023-07-15 12:00:00,30.0,70,28.0,80,18.0,90,20.0,70
"""
dummy_csv_data_ahu2_oac_control = """DATETIME,ZoneA_PREHEAT_온도,ZoneA_PREHEAT_습도,ZoneA_PRECOOL_온도,ZoneA_PRECOOL_습도,MainUnit_C/C_온도_출구,MainUnit_C/C_습도_출구,PostZone_HEAT_온도,PostZone_HEAT_습도
2023-07-16 14:00:00,28.0,70,25.0,80,17.0,88,20.0,65
2023-07-16 14:10:00,28.2,72,24.5,81,16.0,93,22.5,55
2023-07-16 15:00:00,29.0,70,24.0,82,15.5,96,25.0,44
"""
VIRTUAL_FILE_SYSTEM = {
    "data_ahu1_oac.csv": dummy_csv_data_ahu1_oac,
    "data_ahu2_oac_control_DATETIME.csv": dummy_csv_data_ahu2_oac_control,
}

# --- 실행 부분 ---
if __name__ == '__main__':
    use_real_files = input("실제 파일 시스템을 사용하시겠습니까? (y/n, 기본값 n): ").lower() == 'y'
    selected_filename = None
    df_to_process_original = None # 원본 데이터프레임
    internal_timestamp_col = 'Timestamp'

    if use_real_files:
        DATA_DIRECTORY = "./24Y/"
        print(f"지정된 디렉토리: {DATA_DIRECTORY}")
        if not os.path.isdir(DATA_DIRECTORY):
            print(f"오류: 디렉토리를 찾을 수 없습니다: {DATA_DIRECTORY}")
            print(f"현재 작업 디렉토리({os.getcwd()}) 하위에 '24Y' 폴더를 생성하고 CSV 파일을 넣어주세요.")
            exit()
        actual_oac_files = [os.path.join(DATA_DIRECTORY, f) for f in os.listdir(DATA_DIRECTORY) if "oac" in f.lower() and f.endswith(".csv")]
        if not actual_oac_files:
            print(f"디렉토리에서 'OAC' 포함 CSV 파일을 찾을 수 없습니다: {DATA_DIRECTORY}")
            exit()
        print("\n사용 가능한 'OAC' CSV 파일:")
        for i, filepath in enumerate(actual_oac_files): print(f"  {i+1}. {os.path.basename(filepath)}")

        choice = -1
        while not (1 <= choice <= len(actual_oac_files)):
            try: choice = int(input(f"파일 번호를 선택하세요 (1-{len(actual_oac_files)}): "))
            except ValueError: print("잘못된 입력입니다. 숫자를 입력하세요.")
        selected_filepath = actual_oac_files[choice-1]
        selected_filename = os.path.basename(selected_filepath)
        print(f"\n실제 파일에서 데이터 로딩 중: {selected_filename} (대용량 파일은 시간이 소요될 수 있습니다...)")
        try: df_to_process_original = pd.read_csv(selected_filepath)
        except Exception as e: print(f"파일 로딩 오류: {e}"); exit()
    else:
        available_oac_files = find_oac_csv_files_simulated(VIRTUAL_FILE_SYSTEM)
        selected_filename, selected_csv_content = select_csv_file_simulated(available_oac_files)
        if selected_csv_content:
            print(f"\n시뮬레이션 파일에서 데이터 로딩 중: {selected_filename}")
            try: df_to_process_original = pd.read_csv(io.StringIO(selected_csv_content))
            except Exception as e: print(f"시뮬레이션 파일 처리 오류: {e}"); exit()

    if df_to_process_original is not None and selected_filename:
        try:
            # Timestamp 컬럼 표준화
            if 'DATETIME' in df_to_process_original.columns:
                df_to_process_original.rename(columns={'DATETIME': internal_timestamp_col}, inplace=True)
            elif internal_timestamp_col not in df_to_process_original.columns:
                 raise ValueError(f"CSV에 '{internal_timestamp_col}' 또는 'DATETIME' 컬럼이 필요합니다.")
            df_to_process_original[internal_timestamp_col] = pd.to_datetime(df_to_process_original[internal_timestamp_col])

            print(f"원본 데이터 포인트 수: {len(df_to_process_original)}")
            df_indexed = df_to_process_original.set_index(internal_timestamp_col)

            # 리샘플링
            numeric_cols = df_indexed.select_dtypes(include=np.number).columns
            df_resampled = pd.DataFrame()
            if not numeric_cols.empty:
                 valid_numeric_cols = [col for col in numeric_cols if col in df_indexed.columns]
                 if valid_numeric_cols:
                    print("데이터를 1시간 단위 첫 번째 값으로 리샘플링합니다...")
                    df_resampled = df_indexed[valid_numeric_cols].resample('1H').first()
                    df_resampled.dropna(how='all', inplace=True) # 모든 값이 NaN인 행 제거
                 else: # 유효한 숫자형 컬럼이 없는 경우
                    print("경고: 유효한 숫자형 컬럼이 없어 리샘플링을 위한 데이터가 없습니다. 원본 데이터를 사용합니다.")
                    df_resampled = df_indexed.copy()
            else: # 숫자형 컬럼이 없는 경우
                 print("경고: 데이터에 숫자형 컬럼이 없어 리샘플링을 건너<0xEB><0><0x8A>니다. 원본 데이터를 사용합니다.")
                 df_resampled = df_indexed.copy()

            # 리샘플링 후 Timestamp 인덱스가 살아있는지 확인, 없으면 컬럼에서 다시 설정
            if not isinstance(df_resampled.index, pd.DatetimeIndex):
                # 이 경우는 df_indexed.copy()를 사용했거나, resample이 실패하여 인덱스가 사라진 경우일 수 있음
                # df_resampled가 원본 df_indexed와 동일하다면 인덱스는 이미 DatetimeIndex임
                # 만약 reset_index()가 중간에 호출되었다면 다시 set_index 필요
                if internal_timestamp_col in df_resampled.columns:
                    df_resampled[internal_timestamp_col] = pd.to_datetime(df_resampled[internal_timestamp_col])
                    df_resampled = df_resampled.set_index(internal_timestamp_col) # 여기서 df_resampled는 Timestamp를 인덱스로 가짐
                else: # Timestamp 컬럼도 없고, 인덱스도 DatetimeIndex가 아니면 문제
                     print(f"경고: 리샘플링 후 '{internal_timestamp_col}' 인덱스 또는 컬럼을 찾을 수 없습니다.")


            # df_resampled는 이제 Timestamp를 인덱스로 가짐. create_interactive_psychrometric_chart_plotly는 인덱스를 사용함.
            print(f"리샘플링 후 데이터 포인트 수: {len(df_resampled)}")

            if df_resampled.empty:
                print("리샘플링 후 데이터가 없습니다. 차트를 생성할 수 없습니다.")
            else:
                if len(df_resampled) > 1000: # 임계값 조정 (예: 1000 프레임)
                    print(f"주의: 리샘플링 후 데이터 포인트(프레임 수)가 {len(df_resampled)}개로 많습니다. ")
                    print("차트 로딩 및 슬라이더 반응에 시간이 소요될 수 있습니다.")

                # 컬럼 매핑은 리샘플링된 데이터프레임의 컬럼을 사용 (인덱스는 제외)
                mapped_stages = map_columns_from_csv(df_resampled.columns)
                if mapped_stages:
                    # create_interactive_psychrometric_chart_plotly 함수는 Timestamp 인덱스를 가진 df를 기대함
                    create_interactive_psychrometric_chart_plotly(df_resampled, mapped_stages, selected_filename)
                else:
                    print("\n컬럼 매핑 문제로 차트 생성을 진행할 수 없습니다.")
        except Exception as e:
            print(f"파일 {selected_filename} 처리 중 오류 발생: {e}")
            traceback.print_exc()
    elif not selected_filename :
        print("파일이 선택되지 않았거나 시뮬레이션된 파일 내용이 없습니다. 프로그램을 종료합니다.")

실제 파일 시스템을 사용하시겠습니까? (y/n, 기본값 n): n


NameError: name 'find_oac_csv_files_simulated' is not defined